In [20]:
import numpy as np
import json
import os
import collections
import csv
import re
import pickle
import tqdm

from nltk.parse import corenlp
import datetime


In [21]:
# !python --version
# !which python
# !jupyter --version
# !pip -v list

# !pip install jupyterlab_vim

In [22]:
# Jsonファイルの読み込み

filename = "dataset_my-task_2021-09-16_08-10-06-950.json"
dir_name = "log/"
sum_dir_name = "summary/"

with open(filename, "r") as f:
    json_obj = json.load(f)

In [4]:
# review のtextとrating / wordのリスト化

word_2d_list = []
rating_list = []
text_list = []
sentense_list = []
reviewer_loc_list = []
reviewers_cnt_list= []
is_funny_cnt_list = []
is_useful_cnt_list = []
is_cool_cnt_list = []
date_list = []
avoid_word_list = ["(", ")", "!", ",", ":", ";" ,"?", ".", "+", "--", "\"", "/", "#"]


for i in range(len(json_obj[0]["reviews"])):
    word_list = json_obj[0]["reviews"][i]["text"].split()
    text_list.append(json_obj[0]["reviews"][i]["text"])
    sentense_list.append(list(re.split("[.\n!]", json_obj[0]["reviews"][i]["text"])))
    for avoid_word in avoid_word_list:
        word_list = [ x.replace(avoid_word, "").lower() for x in word_list]
    word_list = [x for x in word_list if not x.isdigit() and x != "" and len(x) < 20 and len(x) != 0]
    word_2d_list.append(word_list)
    date_str = json_obj[0]["reviews"][i]["date"]
    date_list.append(datetime.datetime.strptime(date_str, '%Y-%m-%dT%H:%M:%S.000Z'))
    
    rating_list.append(int(json_obj[0]["reviews"][i]["rating"]))
    reviewer_loc_list.append(json_obj[0]["reviews"][i]["reviewerLocation"])
    reviewers_cnt_list.append(json_obj[0]["reviews"][i]["reviewerReviewCount"])
    is_cool_cnt_list.append(json_obj[0]["reviews"][i]["isCoolCount"])
    is_funny_cnt_list.append(json_obj[0]["reviews"][i]["isFunnyCount"])
    is_useful_cnt_list.append(json_obj[0]["reviews"][i]["isUsefulCount"])

In [5]:
# cityを集計して保存

city_cnt = collections.Counter()
city_cnt_after_200101 = collections.Counter()
ref_date = datetime.datetime(2020, 1, 1, 0, 0, 0)

for reviewer_loc, date in zip(reviewer_loc_list, date_list) :
    city_cnt[reviewer_loc.lower()] += 1
    if ref_date < date:
        city_cnt_after_200101[reviewer_loc.lower()] += 1

output_file_name = sum_dir_name + "city_ranking.csv"

city_sorted = sorted(city_cnt.items(), key=lambda x:x[1], reverse=True)
city_a_200101_sorted = sorted(city_cnt_after_200101.items(), key=lambda x:x[1], reverse=True)

with open(output_file_name, "w") as f:
    writer = csv.writer(f)
    writer.writerow(["city 全区間","", "city 20200101〜",""])
    for city, city_after in zip(city_sorted, city_a_200101_sorted):
        writer.writerow([city[0], city[1], city_after[0], city_after[1] ])

In [6]:
# wordを集計して保存

word_cnt = collections.Counter()
ref_date = datetime.datetime(2021, 7, 12, 0, 0, 0)

for word_list, rating, date in zip(word_2d_list, rating_list, date_list):
    if ref_date < date:
        rating_val = rating - 3
        for word in word_list:
            word_cnt[word.lower()] += rating_val

# 星毎に、星の数を集計

word_dict = word_cnt.copy()
# print(word_dict)
for k, v in word_dict.items():
    word_dict[k] = list([v, 0 ,0, 0, 0, 0 ])
#     print(word_dict[k])

for word_list, rating, date in zip(word_2d_list, rating_list, date_list):
    if ref_date < date:
        for word in word_list:
            word_dict[word.lower()][rating ] += 1
    #         if len(word_dict[word.lower()]) <= 6: # 例文をappend済みでなければ
    #             word_dict[word.lower()].append(word_list)

# sortしてファイルに出力

output_file_name = sum_dir_name + "output.csv"

word_dict_sorted = sorted(word_dict.items(), key=lambda x:x[1], reverse=True)

with open(output_file_name, "w") as f:
    writer = csv.writer(f)
    writer.writerow(["word", "review総計" , "星1", "星2", "星3", "星4", "星5"])
    for data_list in word_dict_sorted:
        writer.writerow([ data_list[0], data_list[1][0], data_list[1][1], data_list[1][2], data_list[1][3], data_list[1][4], data_list[1][5] ])

In [7]:
# for i in range(11):
#     print("----------------------")
#     for i, sentense in enumerate(sentense_list[i]):
#         print(i ," : " , sentense)
#         print("")
    # print(text_list[i])
    # print("")

In [8]:
# 文章のdependencyを解析
# 長時間かかるので注意　pickle化してある。

# pickle_file_name = 'dep.pickle'

# dep_parser = corenlp.CoreNLPDependencyParser(url='http://localhost:9000')

# parse_list = []
# parse_2d_list = []

# def sentense_dependency():
#     for i, sentences in tqdm.tqdm(enumerate(sentense_list)):  # 3380のレビューの数 # 1582it エラー
#         # if i < 1581:
#         #     continue
#         for sentence in sentences:  # １レビューあたり入っている文章段落の数
#             if not sentence or len(sentence) < 15:
#                 continue
#             # print("test : ", sentence)
#             parse, = dep_parser.raw_parse(sentence)
#             parse_tmp = parse.to_conll(4).split("\n")
#             parse_2d_list.append([x.split("\t") for x in parse_tmp ])  # if not re.search("[()!,:;?.+-\/#]", x) if x != ""
            
# sentense_dependency()

# with open(pickle_file_name, 'wb') as f:
#     pickle.dump(parse_2d_list, f)
    

In [15]:
# pickle呼び出し

pickle_file_name = 'dep.pickle'

with open(pickle_file_name, 'rb') as f:
    parse_2d_list = pickle.load(f)
    
# for parse_list in parse_2d_list: 
#     print(parse_list)
#     print("")


[['100', 'CD', '2', 'nummod'], ['%', 'NN', '3', 'nsubj'], ['recommend', 'VBP', '0', 'ROOT'], ['getting', 'VBG', '3', 'xcomp'], ['this', 'DT', '4', 'obj'], ['']]
['100', 'CD', '2', 'nummod']
100


In [16]:

# # print(parse_2d_list)
print(parse_2d_list[10])
print(parse_2d_list[10][0])
print(parse_2d_list[10][0][0])
# print("")

[['100', 'CD', '2', 'nummod'], ['%', 'NN', '3', 'nsubj'], ['recommend', 'VBP', '0', 'ROOT'], ['getting', 'VBG', '3', 'xcomp'], ['this', 'DT', '4', 'obj'], ['']]
['100', 'CD', '2', 'nummod']
100


In [20]:

dir_name = "dep_log/"
dep_cnt_list = [collections.Counter() for x in range(len(word_cnt))]

print(len(parse_2d_list))
print(len(parse_2d_list[0]))
print(len(word_cnt.keys()))

# for word_list, sentense, rating, reviewer_loc, reviewers_cnt, is_cool_cnt, is_funny_cnt, is_useful_cnt in zip(word_2d_list, sentense_list, rating_list, reviewer_loc_list, reviewers_cnt_list, is_cool_cnt_list, is_funny_cnt_list, is_useful_cnt_list ):
for word in word_cnt.keys():
    for parse_list in parse_2d_list:
        for parse in parse_list:
            # print(parse[0], word)abs #OK
            if parse[0] == word:
                # print(parse_list[int(parse[2])][0])
                # dep_cnt_list[ ]

<class 'SyntaxError'>: unexpected EOF while parsing (2604225226.py, line 15)

In [7]:
# review textをword毎に集計

output_example_wordlist = ["Wait","Service","Server","Noodle","Togo","Broth","Soup","Host","Appetizer","Bad","Worst","Good","Best","Awesome"]

ref_date = datetime.datetime(2021, 7, 12, 0, 0, 0)

for output_example in output_example_wordlist:
    if os.path.exists(dir_name + output_example.lower() + "_check_" + ".txt"):
        continue
    else:
        with open(dir_name + output_example.lower() + "_check_" + ".txt", "w") as f:
            pass
        for word_list, text, rating, reviewer_loc, reviewers_cnt, is_cool_cnt, is_funny_cnt, is_useful_cnt, date in zip(word_2d_list, text_list, rating_list, reviewer_loc_list, reviewers_cnt_list, is_cool_cnt_list, is_funny_cnt_list, is_useful_cnt_list, date_list ):
        # for word_list, sentense, rating, reviewer_loc, reviewers_cnt, is_cool_cnt, is_funny_cnt, is_useful_cnt in zip(word_2d_list, sentense_list, rating_list, reviewer_loc_list, reviewers_cnt_list, is_cool_cnt_list, is_funny_cnt_list, is_useful_cnt_list ):
            for word in word_list:
                if ref_date < date:
                    if word == output_example.lower():
                        with open(dir_name + word.lower() + "_rating_" + str(rating) + ".txt", "a") as f:
                            f.writelines("date : " + str(date) + "\n")
                            f.writelines("reviewer Location : " + reviewer_loc + "\n")
                            f.writelines("Reviewer Count : " + str(reviewers_cnt) + "\n")
                            f.writelines("is Funny Count : " + str(is_funny_cnt) + "\n")
                            f.writelines("is Usuful Count : " + str(is_useful_cnt) + "\n")
                            f.writelines("is Cool Count : " + str(is_cool_cnt) + "\n\n")
                            f.writelines(text)
                            f.writelines("\n\n===========================================\n\n")
                        break

In [8]:
# # 全ワードのtext列挙

# for word_d in word_dict.keys():
#     if os.path.exists(dir_name + word_d.lower() + "_check_" + ".txt"):
#         continue
#     else:
#         with open(dir_name + word_d.lower() + "_check_" + ".txt", "w") as f:
#             pass 
#     for word_list, text, rating, reviewer_loc, reviewers_cnt, is_cool_cnt, is_funny_cnt, is_useful_cnt in zip(word_2d_list, text_list, rating_list, reviewer_loc_list, reviewers_cnt_list, is_cool_cnt_list, is_funny_cnt_list, is_useful_cnt_list ):
#         for word in word_list:
#             if word == word_d:
#                 with open(dir_name + word.lower() + "_rating_" + str(rating) + ".txt", "a") as f:
#                     f.writelines("reviewer Location : " + reviewer_loc + "\n")
#                     f.writelines("Reviewer Count : " + str(reviewers_cnt) + "\n")
#                     f.writelines("is Funny Count : " + str(is_funny_cnt) + "\n")
#                     f.writelines("is Usuful Count : " + str(is_useful_cnt) + "\n")
#                     f.writelines("is Cool Count : " + str(is_cool_cnt) + "\n\n")
#                     f.writelines(text)
#                     f.writelines("\n\n===========================================\n\n")
#                 break

In [73]:
parser = corenlp.CoreNLPParser(url='http://localhost:9000')
next( parser.raw_parse('The quick brown fox jumps over the lazy dog.') ).pretty_print()  
# next( parser.raw_parse(text_list[5]) ).pretty_print()  

                     ROOT                          
                      |                             
                      S                            
       _______________|__________________________   
      |                         VP               | 
      |                _________|___             |  
      |               |             PP           | 
      |               |     ________|___         |  
      NP              |    |            NP       | 
  ____|__________     |    |     _______|____    |  
 DT   JJ    JJ   NN  VBZ   IN   DT      JJ   NN  . 
 |    |     |    |    |    |    |       |    |   |  
The quick brown fox jumps over the     lazy dog  . 



# 以下ゴミ

In [10]:
# dependency parse with CoreNLP

# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

# https://stanfordnlp.github.io/CoreNLP/download.html

# export CLASSPATH=$CLASSPATH:/home/ted/Downloads/stanford-corenlp-4.2.2/*:

In [11]:
# review　から、文章のlist　その下に wordのlistに分割する処理

# input_review = re.split("[.\n!]", text_list[7])

# input_review_devide_space = [x.split() for x in input_review if x]

# out_review = []
# for sentense in input_review_devide_space:
#     out_sentense = []
#     for word in sentense:
#         re.sub("[(\)\!\,\:\;\?\.\+\--\/\#]", "", word)
#         word = word.lower()
#         if word:
#             out_sentense.append(word)
# #         print(word)
#     if out_sentense:
#         out_review.append(out_sentense)
# print(input_review)


In [12]:
# dep_parser = corenlp.CoreNLPDependencyParser(url='http://localhost:9000')

# for review in input_review:
#     # print(out_s)
#     if not review or review ==" ":
#         continue
#     parse, = dep_parser.raw_parse(review)
#     # 'The quick brown fox jumps over the lazy dog.'
#     print(parse.to_conll(4))

In [3]:
# Natural Language Toolkit

from nltk.corpus import treebank
nltk.download('treebank')

t = treebank.parsed_sents('wsj_0001.mrg')[1]
t.draw()


<class 'NameError'>: name 'nltk' is not defined

In [101]:
# 使えない。

sentence = """At eight o'clock on Thursday morning. Arthur didn't feel very good."""
tokens = nltk.word_tokenize(sentence)
tagged = nltk.pos_tag(tokens)
print(tagged)

[('At', 'IN'), ('eight', 'CD'), ("o'clock", 'NN'), ('on', 'IN'), ('Thursday', 'NNP'), ('morning', 'NN'), ('.', '.'), ('Arthur', 'NNP'), ('did', 'VBD'), ("n't", 'RB'), ('feel', 'VB'), ('very', 'RB'), ('good', 'JJ'), ('.', '.')]


In [2]:
from nltk.grammar import DependencyGrammar
from nltk.parse import (DependencyGraph,ProjectiveDependencyParser,NonprojectiveDependencyParser)
treebank_data = """Pierre  NNP     2       NMOD
Vinken  NNP     8       SUB
,       ,       2       P
61      CD      5       NMOD
years   NNS     6       AMOD
old     JJ      2       NMOD
,       ,       2       P
will    MD      0       ROOT
join    VB      8       VC
the     DT      11      NMOD
board   NN      9       OBJ
as      IN      9       VMOD
a       DT      15      NMOD
nonexecutive    JJ      15      NMOD
director        NN      12      PMOD
Nov.    NNP     9       VMOD
29      CD      16      NMOD
.       .       9       VMOD"""

dg = DependencyGraph(treebank_data)
dg.tree().pprint()

(will
  (Vinken Pierre , (old (years 61)) ,)
  (join (board the) (as (director a nonexecutive)) (Nov. 29) .))
